<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Demo_Blockchain_PoP_(Proof_of_Participants).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json

In [5]:
import hashlib
import time

class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = str(index) + previous_hash + str(timestamp) + data
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", time.time(), "Genesis Block", Block.calculate_hash(0, "0", time.time(), "Genesis Block"))

    def to_dict(self):
        return {
            'index': self.index,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'data': self.data,
            'hash': self.hash
        }

In [6]:
class Blockchain:
    def __init__(self):
        self.chain = [Block.create_genesis_block()]

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            if current_block.hash != Block.calculate_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data):
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True

    def to_json(self):
        chain_data = [block.to_dict() for block in self.chain]
        return json.dumps(chain_data, indent=4)

In [7]:
class Validator:
    def __init__(self, name):
        self.name = name
        self.participation = 0

    def participate(self):
        self.participation += 1

class PoP:
    def __init__(self):
        self.validators = []

    def add_validator(self, validator):
        self.validators.append(validator)

    def select_validator(self):
        # Simple selection based on participation
        max_participation = max([v.participation for v in self.validators])
        for validator in self.validators:
            if validator.participation == max_participation:
                return validator

# Usage Example
blockchain = Blockchain()
pop = PoP()

validator1 = Validator("Validator1")
validator2 = Validator("Validator2")

pop.add_validator(validator1)
pop.add_validator(validator2)

# Simulating participation
validator1.participate()
validator2.participate()
validator2.participate()

selected_validator = pop.select_validator()
print(f"Selected Validator: {selected_validator.name}")

# Adding a new block by selected validator
new_block = Block(len(blockchain.chain), blockchain.get_latest_block().hash, time.time(), "Some Data", "")
blockchain.add_block(new_block)

print("Blockchain is valid:", blockchain.is_chain_valid())

print("Blockchain JSON representation:")
print(blockchain.to_json())

Selected Validator: Validator2
Blockchain is valid: True
Blockchain JSON representation:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1720706237.048032,
        "data": "Genesis Block",
        "hash": "55ff2c29a8b9c32e96d06525fe231fe36671df30e1ffff253d3b8dc868062f51"
    },
    {
        "index": 1,
        "previous_hash": "55ff2c29a8b9c32e96d06525fe231fe36671df30e1ffff253d3b8dc868062f51",
        "timestamp": 1720706237.0501723,
        "data": "Some Data",
        "hash": "e6a3afc067b11e40dc02645a037607484888e8c90f815c05de7b021e1a9d28e1"
    }
]
